In [2]:
import re
from graphviz import Digraph
import json

In [3]:
def validate_regex(regex):
  try:
     re.compile(regex)
     return True
  except re.error:
     return False

In [4]:
def validate_regex2(pattern):
    if not pattern or len(pattern) == 0:
        print("The pattern may not be empty or null.")
        return False

    if "[]" in pattern:
        print("Empty character class.")
        return False

    stack = []
    escaped = False
    inside_square_brackets = False
    prev_char = None

    for i, char in enumerate(pattern):
        if char == '\\':
            escaped = not escaped
        elif char in '(){}' and not escaped:
            if char in '({':
                stack.append(char)
            else:
                if not stack or not is_matching_pair(stack.pop(), char):
                    print(f"Unmatched '{char}'")
                    return False
        elif char == '[' and not escaped:
            inside_square_brackets = True
            stack.append(char)

        elif char == ']' and not escaped:
            inside_square_brackets = False
            if not stack or not is_matching_pair(stack.pop(), char):
                print(f"Unmatched '{char}'")
                return False
            if '[' in stack:
                inside_square_brackets = True
        elif char in ('-', '+', '*') and prev_char == char:
            print(f"Two or more consecutive '{char}'")
            return False
        elif char in ('?', '+', '*') and inside_square_brackets:
            print(f"Operator '{char}' inside [] is invalid")
            return False

        escaped = False
        prev_char = char

    if stack:
        print(f"Unmatched '{stack[-1]}'")
        return False

    return True

def is_matching_pair(left, right):
    pairs = {'(': ')', '[': ']', '{': '}'}
    return pairs[left] == right


In [5]:
regex = "[a-c]+[0--9]*"
is_valid = validate_regex(regex)
print(f"Is valid regex? {is_valid}")


regex = "[a-c"
is_valid = validate_regex(regex)
print(f"Is valid regex? {is_valid}")

regex = "[a-c]"
is_valid = validate_regex(regex)
print(f"Is valid regex? {is_valid}")

regex = "[]"
is_valid = validate_regex(regex)
print(f"Is valid regex? {is_valid}")

regex = "[a-]"
is_valid = validate_regex(regex)
print(f"Is valid regex? {is_valid}")

Is valid regex? False
Is valid regex? False
Is valid regex? True
Is valid regex? False
Is valid regex? True


<ipython-input-3-eef82f8330ac>:3: FutureWarning: Possible set difference at position 8
  re.compile(regex)


In [6]:
def expand_ranges(regex):
    i = 0
    while i < len(regex):
        if regex[i] == '[':
            j = i + 1
            expanded_range = ''
            while j < len(regex) and regex[j] != ']':
                if j < len(regex) - 2 and regex[j + 1] == '-':
                    start = regex[j]
                    end = regex[j + 2]
                    expanded_range += '|'.join(chr(k) for k in range(ord(start), ord(end) + 1))
                    j += 3
                    if j < len(regex) and regex[j] != "]":
                        expanded_range += '|'

                else:
                    expanded_range += regex[j]
                    j += 1
                    if j < len(regex) and regex[j] != "]":
                        expanded_range += '|'

            regex = regex[:i] + '(' + expanded_range + ')' + regex[j + 1:]
        i += 1
    return regex

regex="acv[a-cA-C0-91-3i-lny]"
result=expand_ranges(regex)
print(result)

acv(a|b|c|A|B|C|0|1|2|3|4|5|6|7|8|9|1|2|3|i|j|k|l|n|y)


In [7]:
def concat(regex):
  result=[]
  for i in range(len(regex)):
    result.append(regex[i])
    if(regex[i] in ['*','?','+',')',']'] and (i+1)<len(regex) and regex[i+1] not in ['*','?','+',')',']','.','|']):
      result.append('.')
    elif regex[i].isalnum() and (i+1)<len(regex) and(regex[i+1].isalnum() or regex[i+1] in['(','[']):
      result.append('.')

  return ''.join(result) #==> convert list to string



In [8]:
def test_concat():
    test_cases = [
        ('a', 'a'),
        ('ab', 'a.b'),
        ('a*', 'a*'),
        ('a+b', 'a+.b'),
        ('a?', 'a?'),
        ('(ab)+', '(a.b)+'),
        ('[a-c]', '[a-c]'),
        ('[a-c]*', '[a-c]*'),
        ('[a-c]+', '[a-c]+'),
        ('[a-c]?', '[a-c]?'),
        ('[a-c][d-f]', '[a-c].[d-f]'),
        ('ab(cd)', 'a.b.(c.d)'),
        ('ab(cd)*', 'a.b.(c.d)*'),
        ('ab(cd)?', 'a.b.(c.d)?'),
    ]
    for infix, expected_output in test_cases:
        result = concat(infix)
        assert result == expected_output, f"Failed for infix '{infix}'. Expected '{expected_output}', got '{result}'."

    print("All test cases passed!")

test_concat()

All test cases passed!


In [90]:
def preprocess(infix):
  if validate_regex2(infix):
    preprocessed_infix=expand_ranges(infix)
    preprocessed_infix=concat(preprocessed_infix)
    #print(preprocessed_infix)
    return preprocessed_infix
  else:
    print("Invalid regex")
    return None

In [10]:
preprocess('[a-c]')
preprocess('a*')
preprocess('ab')
preprocess('a+b')
preprocess('(ab)+')
preprocess('[a-c]*')
preprocess('[a-c]?')
preprocess('[a-c][d-f]')
preprocess('ab(cd)')
preprocess('ab(cd)*')
preprocess('ab(cd)?')
preprocess('ab(cd)?')
preprocess('ab(cd)?')
preprocess('[a-z')
preprocess('[a-z')
preprocess('ab)')
preprocess('a?*')
preprocess('a(b|c)')
preprocess('a(b|a)*b')

(a|b|c)
a*
a.b
a+.b
(a.b)+
(a|b|c)*
(a|b|c)?
(a|b|c).(d|e|f)
a.b.(c.d)
a.b.(c.d)*
a.b.(c.d)?
a.b.(c.d)?
a.b.(c.d)?
Unmatched '['
Invalid regex
Unmatched '['
Invalid regex
Unmatched ')'
Invalid regex
a?*
a.(b|c)
a.(b|a)*.b


'a.(b|a)*.b'

In [11]:
def shunting_yard_algo(infix):
  stack=[]
  postfix=[]
  operators={'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}

  for token in infix:
    if (token=='('):
      stack.append(token)
    elif(token.isalnum()):
      postfix.append(token)
    elif(token in operators):
      while stack and stack[-1] in operators and operators[stack[-1]]>=operators[token]:
        postfix.append(stack.pop())
      stack.append(token)

    elif(token==')'):
      while stack and stack[-1]!='(':
        postfix.append(stack.pop())
      if stack and stack[-1]=='(':
        stack.pop()

  while stack:
    postfix.append(stack.pop())

  postfix=''.join(postfix)
  return postfix


In [12]:
postfix=shunting_yard_algo("(A+.B*)?.(C|D)")
print(postfix)

A+B*.?CD|.


In [13]:
def test_shunting_yard_algo():
    test_cases = [
        ('a.b|c*', 'ab.c*|'),
        ('a|b.c', 'abc.|'),
        ('a.(b|c)*.d', 'abc|*.d.'),
        ('a.b.c|d*', 'ab.c.d*|'),
        ('a|(b.c)*', 'abc.*|'),
        ('a.(b.c)*.d', 'abc.*.d.'),
    ]

    for infix, expected_output in test_cases:
        result = shunting_yard_algo(infix)
        assert result == expected_output, f"Failed for infix '{infix}'. Expected '{expected_output}', got '{result}'."

    print("All test cases passed!")


test_shunting_yard_algo()


All test cases passed!


In [14]:
regex="a*(a|b)aa"
processed_regex=preprocess(regex)
if processed_regex:
  print(processed_regex)
  postfix=shunting_yard_algo(processed_regex)
  print(postfix)

a*.(a|b).a.a
a*.(a|b).a.a
a*ab|.a.a.


In [15]:
class NFAState:
    def __init__(self, name, is_terminating=False):
        self.name = name
        self.is_terminating = is_terminating
        self.transitions = {}

    def add_transition(self, symbol, state):
        if symbol in self.transitions:
            self.transitions[symbol].append(state)
        else:
            self.transitions[symbol] = [state]




class NFATransition:
    def __init__(self, start_state, symbol, end_state):
        self.start_state = start_state
        self.symbol = symbol
        self.end_state = end_state


class NFA:
    def __init__(self):
        self.states = {}
        self.state_count = -1

    def add_state(self, is_terminating=False):
        self.state_count += 1
        state_name = f"S{self.state_count}"
        state = NFAState(state_name, is_terminating)
        self.states[state_name] = state
        return state

    def add_transition(self, start_state, symbol, end_state):
        transition = NFATransition(start_state, symbol, end_state)
        start_state.add_transition(symbol, end_state)


def thompson_construction(regex):
    nfa = NFA()
    stack = []

    class NFAStatePair:
        def __init__(self, start_state, current_state):
            self.start_state = start_state
            self.current_state = current_state

    for token in regex:

        if token == '|':
          right = stack.pop()
          left = stack.pop()
          new_start = nfa.add_state()
          new_end = nfa.add_state(is_terminating=False)
          nfa.add_transition(new_start, 'ε', left.start_state)
          nfa.add_transition(new_start, 'ε', right.start_state)
          nfa.add_transition(left.current_state, 'ε', new_end)
          nfa.add_transition(right.current_state, 'ε', new_end)
          stack.append(NFAStatePair(new_start, new_end))

        elif token == '.':
          right = stack.pop()
          left = stack.pop()
          nfa.add_transition(left.current_state, 'ε', right.start_state)
          stack.append(NFAStatePair(left.start_state, right.current_state))

        elif token == '*':
          state_pair = stack.pop()
          new_start = nfa.add_state()
          new_end = nfa.add_state(is_terminating=False)
          nfa.add_transition(new_start, 'ε', state_pair.start_state)
          nfa.add_transition(new_start, 'ε', new_end)
          nfa.add_transition(state_pair.current_state, 'ε', new_end)
          nfa.add_transition(state_pair.current_state, 'ε', state_pair.start_state)
          stack.append(NFAStatePair(new_start, new_end))

        elif token == '?':
          state_pair = stack.pop()
          new_start = nfa.add_state()
          new_end = nfa.add_state(is_terminating=False)
          nfa.add_transition(new_start, 'ε', state_pair.start_state)
          nfa.add_transition(new_start, 'ε', new_end)
          nfa.add_transition(state_pair.current_state, 'ε', new_end)
          stack.append(NFAStatePair(new_start, new_end))

        elif token == '+':
          state_pair = stack.pop()
          new_start = nfa.add_state()
          new_end = nfa.add_state(is_terminating=False)
          nfa.add_transition(new_start, 'ε', state_pair.start_state)
          nfa.add_transition(state_pair.current_state, 'ε', new_end)
          nfa.add_transition(state_pair.current_state, 'ε', state_pair.start_state)
          stack.append(NFAStatePair(new_start, new_end))

        else:
            start_state = nfa.add_state()
            current_state = nfa.add_state(is_terminating=False)
            nfa.add_transition(start_state, token, current_state)
            stack.append(NFAStatePair(start_state, current_state))

    stack[-1].current_state.is_terminating = True
    nfa.start_state = stack[0].start_state
    return nfa

def nfa_to_json(nfa):
    json_data = {
        "startingState": nfa.start_state.name,
    }
    for state_name, state in nfa.states.items():
        transitions = {}
        for symbol, next_states in state.transitions.items():
            next_state_names = [next_state.name for next_state in next_states]
            if len(next_state_names)==1:
              transitions[symbol] = next_state_names[0]
            else:
              transitions[symbol] = next_state_names
        json_data[state_name] = {
            "isTerminatingState": state.is_terminating,
            **transitions
        }
    return json.dumps(json_data, indent=4, ensure_ascii = False)

def nfa_to_graph(nfa):
    graph = Digraph(graph_attr={'rankdir':'LR'})
    for state_name, state in nfa.states.items():
        if state.is_terminating:
            graph.node(state_name, shape='doublecircle')
        if state_name == nfa.start_state.name:
            graph.node("", _attributes= {'shape': 'none'})
            graph.edge("",state_name)
        else:
            graph.node(state_name)
        for symbol, next_states in state.transitions.items():
            for next_state in next_states:
                graph.edge(state_name, next_state.name, label=symbol)
    return graph

In [74]:
class DFAState:
    def __init__(self, name, is_terminating=False):
        self.name = name
        self.is_terminating = is_terminating
        self.transitions = {}

    def add_transition(self, symbol, state):
        if symbol in self.transitions:
            self.transitions[symbol].append(state)
        else:
            self.transitions[symbol] = [state]


class DFATransition:
    def __init__(self, start_state, symbol, end_state):
        self.start_state = start_state
        self.symbol = symbol
        self.end_state = end_state


class DFA:
    def __init__(self):
        self.states = {}
        self.state_count = -1
        self.start_state = None

    def add_state(self, is_terminating=False):
        self.state_count += 1
        state_name = f"S{self.state_count}"
        state = DFAState(state_name, is_terminating)
        self.states[state_name] = state
        return state

    def add_transition(self, start_state, symbol, end_state):
        transition = DFATransition(start_state, symbol, end_state)
        start_state.add_transition(symbol, end_state)

    def set_start_state(self, state):
        self.start_state = state

In [75]:
def get_epsilon_closure(state, data, epsilon_closure, visited_states):
  if state not in visited_states:
    visited_states.add(state)
    is_terminating = False
    if "ε" in data[state]:
      epsilon_moves = data[state]["ε"]
      if isinstance(epsilon_moves, str):
        epsilon_closure.add(epsilon_moves)
        if(data[epsilon_moves]["isTerminatingState"]):
          is_terminating = True
        is_terminating_for_state = get_epsilon_closure(epsilon_moves, data, epsilon_closure, visited_states)
        is_terminating = is_terminating or is_terminating_for_state
      else:
        for next_state in epsilon_moves:
          epsilon_closure.add(next_state)
          is_terminating_for_state = data[next_state]["isTerminatingState"]
          is_terminating_for_state_2 = get_epsilon_closure(next_state, data, epsilon_closure, visited_states)
          is_terminating_for_state = is_terminating_for_state or is_terminating_for_state_2
          is_terminating = is_terminating or is_terminating_for_state
    return is_terminating

In [76]:
def get_all_transitions(states, data):
  transitions = set()
  for state in states:
    transitions.update(key for key in data[state] if key != "isTerminatingState" and key != "ε")
  return transitions

In [77]:
processed = 0
def complete_dfa(epsilon_closure, data, last_state, dfa):
  global processed
  processed += 1
  transitions = get_all_transitions(epsilon_closure, data)
  for transition in transitions:
    new_closure = set()
    is_terminating = False
    for state in epsilon_closure:
      if transition in data[state]:
        if isinstance(data[state][transition], str):
          new_closure.add(data[state][transition])
        else:
          for transition_state in data[state][transition]:
            new_closure.add(transition_state)
    new_epsilon_closure = new_closure.copy()
    is_terminating = False
    for state in new_closure:
      is_terminating_for_state = data[state]["isTerminatingState"]
      visited_states = set()
      is_terminating_for_state_2 = get_epsilon_closure(state, data, new_epsilon_closure, visited_states)
      is_terminating = is_terminating_for_state or is_terminating_for_state_2
    if frozenset(new_epsilon_closure) not in dfa_states_with_status:
      new_state = dfa.add_state(is_terminating)
      dfa_states_with_status[frozenset(new_epsilon_closure)] = (new_state,is_terminating)
    else:
      new_state = dfa_states_with_status[frozenset(new_epsilon_closure)][0]
    dfa.add_transition(last_state, transition, new_state)

In [78]:
def count_letter_F(matrix):
    count = 0
    for row in matrix:
        for cell in row:
            if cell == 'F':
                count += 1
    return count

In [79]:
def dfa_to_json(dfa):
    json_data = {
        "startingState": dfa.start_state.name,
    }
    for state_name, state in dfa.states.items():
        transitions = {}
        for symbol, next_states in state.transitions.items():
            next_state_names = [next_state.name for next_state in next_states]
            if len(next_state_names)==1:
              transitions[symbol] = next_state_names[0]
            else:
              transitions[symbol] = next_state_names
        json_data[state_name] = {
            "isTerminatingState": state.is_terminating,
            **transitions
        }
    return json.dumps(json_data, indent=4, ensure_ascii = False)

def dfa_to_graph(dfa):
    graph = Digraph(graph_attr={'rankdir':'LR'})
    for state_name, state in dfa.states.items():
        if state.is_terminating:
            graph.node(state_name, shape='doublecircle')
        if state_name == dfa.start_state.name:
            graph.node("", _attributes= {'shape': 'none'})
            graph.edge("",state_name)
        else:
            graph.node(state_name)
        for symbol, next_states in state.transitions.items():
            for next_state in next_states:
                graph.edge(state_name, next_state.name, label=symbol)
    return graph

In [88]:
def min_DFA(dfa, data):
  n_states = dfa.state_count + 1
  matrix = [[0] * n_states for _ in range(n_states)]
  # if one of them is terminal state and the other isn't terminal
  for state_name_1, state_1 in dfa.states.items():
    for state_name_2, state_2 in dfa.states.items():
      if state_1 != state_2:
          if ((state_1.is_terminating and not state_2.is_terminating)
          or (state_2.is_terminating and not state_1.is_terminating)) :
            matrix[int(state_name_1[1:])][int(state_name_2[1:])] = 'F'
            matrix[int(state_name_2[1:])][int(state_name_1[1:])] = 'F'
  # loop over all state pairs, check if an input lead to a different group ==> mark the pair as a different group
  while (True):
    count_before = count_letter_F(matrix)
    for state_name_1, state_1 in dfa.states.items():
      for state_name_2, state_2 in dfa.states.items():
        if state_1 != state_2:
          if(matrix[int(state_name_1[1:])][int(state_name_2[1:])] != 'F'):
            states_set = {state_name_1, state_name_2}
            transitions = get_all_transitions(states_set, data)
            for transition in transitions:
              if transition in data[state_name_1] and transition in data[state_name_2]:
                group_1 = data[state_name_1][transition]
                group_2 = data[state_name_2][transition]
                if matrix[int(group_1[1:])][int(group_2[1:])] == 'F':
                  matrix[int(state_name_1[1:])][int(state_name_2[1:])] = 'F'
                  matrix[int(state_name_2[1:])][int(state_name_1[1:])] = 'F'
              elif transition in data[state_name_1] or  transition in data[state_name_2]:
                matrix[int(state_name_1[1:])][int(state_name_2[1:])] = 'F'
                matrix[int(state_name_2[1:])][int(state_name_1[1:])] = 'F'
    count_after = count_letter_F(matrix)
    if count_before == count_after:
      break
  return matrix

In [81]:
def merge_sets(set_of_sets):
    merged_sets = set()

    for subset in set_of_sets:
        # Check if any of the existing merged sets have common elements with the current subset
        merged_with_existing = False
        for merged_set in merged_sets.copy():
            if merged_set.intersection(subset):
                merged_sets.remove(merged_set)
                merged_sets.add(frozenset(merged_set.union(subset)))
                merged_with_existing = True
                break

        # If the current subset has no common elements with any existing merged sets, add it as a new merged set
        if not merged_with_existing:
            merged_sets.add(subset)

    # Remove subsets that have been merged into larger sets
    final_sets = set()
    for merged_set in merged_sets:
        is_subset = False
        for other_set in merged_sets:
            if merged_set != other_set and merged_set.issubset(other_set):
                is_subset = True
                break
        if not is_subset:
            final_sets.add(merged_set)

    return final_sets

In [91]:
# output cell
regex = input("Enter a regular expression: ")
preprocessed = preprocess(regex)
if preprocessed:
  #print(preprocessed)
  if preprocessed != None:
    postfix=shunting_yard_algo(preprocessed)
    #print(postfix)
    nfa=thompson_construction(postfix)
    json_data = nfa_to_json(nfa)
    print("NFA json")
    print(json_data)
    graph = nfa_to_graph(nfa)
    graph.render('nfa_graph', format='png', cleanup=True)
    print("NFA graph image saved as 'nfa_graph.png'")

  processed = 0

  dfa = DFA()
  dfa_states_with_status = {}
  # Read the json datastate_string
  data = json.loads(json_data)

  # Get the starting state
  starting_state = data["startingState"]
  # Step 1: Get start state “reachable through ε-moves”
  epsilon_closure = set()
  epsilon_closure.add(starting_state)

  visited_states = set()
  is_terminating = get_epsilon_closure(starting_state, data, epsilon_closure, visited_states)

  start_state = dfa.add_state(is_terminating)
  dfa.set_start_state(start_state)
  dfa_states_with_status[frozenset(epsilon_closure)] = (start_state,is_terminating)

  number_of_states_before = 0
  number_of_states_after = len(dfa_states_with_status)

  keys_to_iterate = list(dfa_states_with_status.keys())
  i = 0
  while(True):
    number_of_states_before = len(dfa_states_with_status)
    complete_dfa(keys_to_iterate[i], data, dfa_states_with_status[keys_to_iterate[i]][0], dfa)
    i = i + 1
    keys_to_iterate = list(dfa_states_with_status.keys())
    number_of_states_after = len(dfa_states_with_status)
    if (number_of_states_before == number_of_states_after and processed == number_of_states_after):
      break

  dfa_json_data = dfa_to_json(dfa)
  print('-----------------------------------------------')
  print("DFA json")
  print(dfa_json_data)
  graph = dfa_to_graph(dfa)
  graph.render('dfa_graph', format='png', cleanup=True)
  print("DFA graph image saved as 'dfa_graph.png'")

  data = json.loads(dfa_json_data)
  matrix = min_DFA(dfa, data)

  min_dfa_sets = set()
  # Iterate over the upper triangular part as it's a symmetric matrix
  # combine state numbers that are in the same group in sets
  for i in range(len(matrix)):
    row_set = set()
    row_set.add(i)
    for j in range(i + 1, len(matrix)):
      if i != j and matrix[i][j] == 0:
        # states than will be combined
        row_set.add(j)
    min_dfa_sets.add(frozenset(row_set))

  final_sets = merge_sets(min_dfa_sets)

  # build new dfa
  # create the states
  dfa = DFA()

  # get the index of the start state
  start_state_index = data['startingState'][1:]

  states_list = []
  for subset in final_sets:
    # loop on sets and create states
    is_terminating = False
    for index in subset:
      element = 'S' + str(index)
      if(data[element]['isTerminatingState']):
        is_terminating = True
        break
    new_state = dfa.add_state(is_terminating)
    if int(start_state_index) in subset:
      # mark start state
      dfa.set_start_state(new_state)
    states_list.append((subset, new_state))

  # create the transitions
  for subset, state in states_list:
    new_set = set()
    for element in subset:
      state_name = 'S' + str(element)
      new_set.add(state_name)
    transitions = get_all_transitions(new_set, data)
    for transition in transitions:

      # Access the first element
      my_list = list(subset)
      element_from_the_set = my_list[0]

      state_name = 'S' + str(element_from_the_set)
      if transition in data[state_name]:
        group = data[state_name][transition]
        # get the end_state of the transition
        for end_subset, end_state in states_list:
          if int(group[1:]) in end_subset:
            end_transition_state = end_state
            break
        dfa.add_transition(end_state=end_transition_state,symbol= transition,start_state = state)

  min_dfa_json_data = dfa_to_json(dfa)
  print('-----------------------------------------------')
  print("Minimized DFA json")
  print(min_dfa_json_data)
  graph = dfa_to_graph(dfa)
  graph.render('min_dfa_graph', format='png', cleanup=True)
  print("Minimized DFA graph image saved as 'min_dfa_graph.png'")

Enter a regular expression: a*b+[a-z](c?)
NFA json
{
    "startingState": "S2",
    "S0": {
        "isTerminatingState": false,
        "a": "S1"
    },
    "S1": {
        "isTerminatingState": false,
        "ε": [
            "S3",
            "S0"
        ]
    },
    "S2": {
        "isTerminatingState": false,
        "ε": [
            "S0",
            "S3"
        ]
    },
    "S3": {
        "isTerminatingState": false,
        "ε": "S6"
    },
    "S4": {
        "isTerminatingState": false,
        "b": "S5"
    },
    "S5": {
        "isTerminatingState": false,
        "ε": [
            "S7",
            "S4"
        ]
    },
    "S6": {
        "isTerminatingState": false,
        "ε": "S4"
    },
    "S7": {
        "isTerminatingState": false,
        "ε": "S108"
    },
    "S8": {
        "isTerminatingState": false,
        "a": "S9"
    },
    "S9": {
        "isTerminatingState": false,
        "ε": "S13"
    },
    "S10": {
        "isTerminatingState": false,
 